# 문자열 데이터 전처리

## 워드 임베딩(Word Embedding)

컴퓨터가 자연어를 이해하고, 효율적으로 처리하게 하기 위해서는 컴퓨터가 이해할 수 있도록 자연어를 적절히 변환할 필요가 있다.

워드 임베딩(Word Embedding)은 단어를 벡터로 표현하는 방법으로, 단어를 희소 표현이 개선된 형태인 밀집 표현으로 변환한다.

### 1. 희소 표현 (원-핫 백터)

표현하고자 하는 단어의 인덱스 값만 1이고 나머지 인덱스에는 전부 0으로 표현되는 백터 방법으로 원-핫 백터라고도 한다.

> `G-01-(2)-09-비정형데이터-분류` 예제에서 소개한 **문서단위행렬**

#### 예시문장 두 개

```
머신러닝 공부는 재미있다.
머신러닝은 유용하다.
```

#### 형태소 분석

위 의 두 예문에서 명사인 단어만 추출한다면 다음과 같다.

```
머신러닝, 공부, 재미
머신러닝, 유용
```

#### 토큰화

각 단어에 인덱스 번호를 적용한다면 아래와 같이 표현할 수 있을 것이다.

```
0, 1, 2
0, 3
```

#### 희소 표현

전체 단어의 수가 `4`개 이므로 각각의 단어를 4개의 원소를 갖는 리스트 안에서 one-hot 인코딩으로 표현한다.

```
[ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0] ]
[ [1, 0, 0, 0], [0, 0, 0, 1] ]
```

#### 희소 표현의 한계

단어의 개수가 늘어나면 벡터의 차원이 한없이 커진다.

여러 문장을 통해 얻어진 단어가 10,000개이고 그 중에서 5개의 단어로 구성된 문장이라면 벡터의 총 길이는 50,000이 되게 된다.

그러므로 공간적인 낭비를 가져온다.


### 2. 밀집표현

희소 표현의 반대

벡터의 차원을 단어 집합의 크기로 결정하지 않고 분석가가 설정한 임의의 값으로 모든 단어 벡터의 차원을 맞춘다.

`0`과 `1`이 아닌 실수를 원소로 갖는다.

### 앞에서 제시한 예시 문장에서 `머신러닝`이라는 단어의 예시

희소표현 → `[1, 0, 0, 0]`

벡터의 차원을 2로 설정한 밀집표현 → `[1.8, -0.4]`

> 수치 값은 임의의 값이다. 이와 같은 식으로 실수 형태로 표현된다는 것을 표현한 것일 뿐 실제 정확한 값은 아님을 이해하자.

결과적으로 벡터의 차원이 조밀해졌다고 하여 밀집 벡터라고 부른다.

### 3. 워드 임베딩

단어를 밀집 벡터의 형태로 표현한 방법.

케라스에서는 **토큰화** 한 단어 벡터를 `Embedding()` 함수에 전달하여 학습층을 쌓음으로서 적용할 수 있다.

단어를 랜덤한 값을 갖는 밀집 벡터로 변환한 뒤에, 인공 신경망의 가중치를 학습하는 방식으로 단어 벡터를 학습한다.


---

## #01. 준비작업

### [1] 패키지 가져오기

In [1]:
# 연결된 모듈이 업데이트 되면 즉시 자동 로드함
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings(action="ignore")

from hossam.util import *
from hossam.plot import *
from hossam.tensor import *

import requests

# 형태소 분석 엔진 -> Okt
# from konlpy.tag import Okt

# 형태소 분석 엔진 -> Mecab
from konlpy.tag import Mecab

# 문자열 토큰화 처리
from tensorflow.keras.preprocessing.text import Tokenizer

[scatterd] >WARNING> From c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.



## #02. 문자열 전처리

### [1] 문자열 토큰화

학습 데이터에서 단어단위로 일련번호로 변환하는 처리

#### (1) 토큰화 학습 데이터

In [2]:
train_text = ["You are the Best", "You are the Nice"]

#### (2) 토큰화 객체 생성

- `num_words` : 밀집 표현의 최대 벡터 길이를 지정 (= 최대 단어 수)
- `oov_token` : 학습 결과를 적용할 때 학습 데이터에 포함되지 않은 단어가 존재할 경우 대체할 단어.

In [3]:
tokenizer = Tokenizer(num_words=10, oov_token="<OOV>")
tokenizer

#### (3) 토큰화 학습하기

In [4]:
# 토큰화 학습
tokenizer.fit_on_texts(train_text)

# 각 단어에 부여된 인덱스 번호 확인
print(tokenizer.word_index)

{'<OOV>': 1, 'you': 2, 'are': 3, 'the': 4, 'best': 5, 'nice': 6}


#### 학습 결과 적용하기

토큰화를 학습한 모델에 새로운 단어를 적용하여 밀집 벡터로 변환한다.

In [5]:
train_text = ["We are the Best", "We are the Nice"]
sequences = tokenizer.texts_to_sequences(train_text)
print(sequences)

[[1, 3, 4, 5], [1, 3, 4, 6]]


### [2] 한글 문장 토큰화

#### (1) 토큰화 학습 데이터

In [6]:
poem = """
흘러내린 머리카락이 흐린 호박빛 아래 빛난다.
난 유영한다. 차분하게 과거에 살면서 현재의 공기를 마신다.
가로등이 깜빡인다.
나도 깜빡여준다.
머리카락이 흩날린다.
"""

#### (2) 불용어(stopwords) 목록

분석에서 제외할 불필요한 단어에 대한 목록.

개발자가 임의로 정하거나 불용어 사전을 웹에서 내려받아 사용할 수 있다.

> 한글 불용어 리스트 출처: https://gist.github.com/spikeekips/40eea22ef4a89f629abd87eed535ac6a

In [7]:
session = requests.Session()
session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

stopwords= None

try:
    r = session.get("https://data.hossam.kr/tmdata/stopwords-ko.txt")

    # HTTP 상태값이 200이 아닌 경우는 에러로 간주한다.
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
    
    r.encoding = "utf-8"
    stopwords = r.text.split("\n")
except Exception as e:
    print(e)

print(stopwords)

['가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '같다', '같이', '개의치않고', '거니와', '거바', '거의', '것', '것과 같이', '것들', '게다가', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '고려하면', '고로', '곧', '공동으로', '과', '과연', '관계가 있다', '관계없이', '관련이 있다', '관하여', '관한', '관해서는', '구', '구체적으로', '구토하다', '그', '그들', '그때', '그래', '그래도', '그래서', '그러나', '그러니', '그러니까', '그러면', '그러므로', '그러한즉', '그런 까닭에', '그런데', '그런즉', '그럼', '그럼에도 불구하고', '그렇게 함으로써', '그렇지', '그렇지 않다면', '그렇지 않으면', '그렇지만', '그렇지않으면', '그리고', '그리하여', '그만이다', '그에 따르는', '그위에', '그저', '그중에서', '그치지 않다', '근거로', '근거하여', '기대여', '기점으로', '기준으로', '기타', '까닭으로', '까악', '까지', '까지 미치다', '까지도', '꽈당', '끙끙', '끼익', '나', '나머지는', '남들', '남짓', '너', '너희', '너희들', '네', '넷', '년', '논하지 않다', '놀라다', '누가 알겠는가', '누구', '다른', '다른 방면으로', '다만', '다섯', '다소', '다수', '다시 말하자면', '다시말하면', '다음', '다음에', '다음으로', '단지', '답다', '당신', '당장', '대로 하다', '대하면', '대하여', '대해 말하자면', '대해서', '댕그', '더구나', '더군다나', '더라도', '더불어', '더욱더', '더욱이는', '도달하다', '도착하다', '동시에', '동안', '된바에야', '된이상', '두번째로', '둘', '둥둥', '뒤따라', '뒤이어'

#### (3) Token (=형태소) 분리

> `D-05-02-워드클라우드(한글)` 예제에서 형태소 분석에 대해 자세히 소개하고 있습니다.

- 형태소란 문법적으로 더 이상 나눌 수 없는 언어 요소.
- 형태소 분리는 단어를 품사별로 구별하는 것을 말한다.
- 영어의 경우 각 단어로 나누면 되지만 한글의 경우 복잡한 처리 과정을 거쳐야 하기 때문에 별도의 라이브러리를 적용해야 한다. (konlpy, mecab 등)
- 머신러닝에서는 문장에서 명사만을 추출하는 것을 목표로 한다.

In [8]:
if sys.platform == "win32":
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
else:
    mecab = Mecab()
    
nouns = mecab.nouns(poem)
print(nouns)

['머리카락', '호박', '빛', '아래', '난', '유영', '과거', '현재', '공기', '가로등', '나', '머리카락']


#### (4) 추출된 명사에서 불용어를 제외한 새로운 리스트 만들기

In [9]:
train_text = [x for x in nouns if x not in stopwords]
train_text

['머리카락', '호박', '빛', '아래', '난', '유영', '과거', '현재', '공기', '가로등', '머리카락']

#### (5) 토큰화 수행

In [10]:
tokenizer = Tokenizer(num_words=len(nouns), oov_token="<OOV>")
tokenizer.fit_on_texts(train_text)
print(tokenizer.word_index)

{'<OOV>': 1, '머리카락': 2, '호박': 3, '빛': 4, '아래': 5, '난': 6, '유영': 7, '과거': 8, '현재': 9, '공기': 10, '가로등': 11}


## #03. 모듈화 기능 확인

### [1] 불용어 목록 생성

In [11]:
stopwords = my_stopwords()
stopwords

['가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 '댕그',


### [2] 한글 형태소 분석

In [12]:
n = my_text_morph("안녕하세요. 반갑습니다. 오늘은 날씨가 좋네요.", stopwords, dicpath="C:\\mecab\\mecab-ko-dic")
n

['안녕', '날씨']

### [3] 토큰 생성

In [13]:
token = my_tokenizer(n)
token

In [14]:
token.word_index

{'<OOV>': 1, '안녕': 2, '날씨': 3}

In [15]:
token.word_counts

OrderedDict([('안녕', 1), ('날씨', 1)])